<a href="https://colab.research.google.com/github/RahulParajuli/NLP-Chatbot-NLTK/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import random
import json
import pickle
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download("omw-1.4")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [39]:
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [40]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())

In [41]:
words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

In [42]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [43]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words)) #to reduce duplicate elements


In [44]:
classes = sorted(set(classes))

In [45]:
print(words)

["'s", 'Bye', 'Do', 'Good', 'Goodbye', 'Hello', 'Hey', 'Hi', 'How', 'Is', 'Mastercard', 'See', 'Thank', 'Thanks', 'That', 'VISA', 'What', 'When', 'Where', 'a', 'accept', 'address', 'anyone', 'are', 'card', 'cost', 'credit', 'day', 'doe', 'have', 'helpful', 'hour', 'it', 'later', 'located', 'location', 'moped', 'motorcycle', 'much', 'open', 'physical', 'rent', 'scooter', 'take', 'there', 'to', 'you', 'your']


In [46]:
print(classes)

['goodbye', 'greeting', 'hours', 'location', 'mopeds', 'payments', 'rental', 'thanks']


In [47]:
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [48]:
training = []
output_empty = [0] * len(classes)

In [49]:
for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1

    training.append([bag, output_row])

In [50]:
random.shuffle(training)
training = np.array(training)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [51]:
train_x = list(training[:, 0])
train_y = list(training[:, 1])

In [52]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [53]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [54]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)


Epoch 1/200
6/6 [==============================] - 0s 3ms/step - loss: 2.0945 - accuracy: 0.1852
Epoch 2/200
6/6 [==============================] - 0s 2ms/step - loss: 2.1122 - accuracy: 0.1852
Epoch 3/200
6/6 [==============================] - 0s 2ms/step - loss: 2.0216 - accuracy: 0.2222
Epoch 4/200
6/6 [==============================] - 0s 3ms/step - loss: 2.0308 - accuracy: 0.2222
Epoch 5/200
6/6 [==============================] - 0s 4ms/step - loss: 1.8924 - accuracy: 0.3333
Epoch 6/200
6/6 [==============================] - 0s 2ms/step - loss: 1.7913 - accuracy: 0.4444
Epoch 7/200
6/6 [==============================] - 0s 3ms/step - loss: 1.7694 - accuracy: 0.3333
Epoch 8/200
6/6 [==============================] - 0s 2ms/step - loss: 1.7316 - accuracy: 0.4074
Epoch 9/200
6/6 [==============================] - 0s 2ms/step - loss: 1.7619 - accuracy: 0.4074
Epoch 10/200
6/6 [==============================] - 0s 2ms/step - loss: 1.5780 - accuracy: 0.4815
Epoch 11/200
6/6 [===========

# chat

In [55]:
from tensorflow.keras.models import load_model

In [56]:
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbot_model.h5')

In [57]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [58]:
def bag_of_words(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, word in enumerate(words):
            if word == s:
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % word)
    return(np.array(bag))

In [68]:
def predict_class(sentence, model):
    p = bag_of_words(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [69]:
predict_class("when are you open?", model)

[{'intent': 'hours', 'probability': '0.999506'}]